In [1]:
import os
import pandas as pd 
import numpy as np
import os, shutil, glob
from random import randint
import re, math
import datetime
import gc
from pathlib import Path

os.getcwd()


'/scratch2-shared/david/spherebooks'

### Read data

In [2]:
pd.set_option("max_colwidth", 2000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)


BaseDir = '/share/data/analyses/christa/colopaint3D/pilots1-10/results_spheroid'
sorted(os.listdir(BaseDir))

['featICF_Experiment.csv',
 'featICF_Image.csv',
 'featICF_LargeBrightArtefacts_enlarge.csv',
 'featICF_LargeBrightArtefacts_identify.csv',
 'featICF_cells.csv',
 'featICF_cytoplasm.csv',
 'featICF_nuclei.csv',
 'featICF_precells.csv',
 'featICF_prenuclei.csv']

### Input directories

In [3]:
# InputFolders = []
# NameContains = input("Enter part of name(s) of the folders: ") # e.g.: TX-.*GR-v2
# for f in (f for f in os.listdir(BaseDir) if re.match(NameContains, f)):
#     InputFolders.append(f)
# InputFolders.sort()
# print ('InputFolders: ' + str(InputFolders))
OutputDir = 'output/2_FIM'
if not os.path.exists(OutputDir): 
    os.makedirs(OutputDir)
NameContains = ''
InputFolders = pd.read_csv('./settings/filemap.csv')
print(InputFolders )
# drugmap = pd.read_csv('settings/drug-well-map.csv', sep=',')
# drugmap.reset_index(inplace=True)
# InputFolders.sort_values(['time-point'], ignore_index=True, inplace=True)
# InputFolders['time-point']
# test = pd.read_parquet('output/2_FIM/ImageMeansAllFrames.parquet')
# for indF,inpF in InputFolders.iterrows():
#     ObjectList = ['featICF_nuclei', 'featICF_cells', 'featICF_cytoplasm']
#     for obj in ObjectList:
#         feature_files = glob.glob(BaseDir + '/' + str(inpF['plate-id']) + '/' + str(inpF[' feat-id']) + '/' + obj + '.csv', recursive=True)
#         # print(feature_files)
#         for file in feature_files[-1:]:
#             print(file)
#             print(file)
#             DataFromOneFile =  pd.read_csv(file, sep=",", low_memory=False)
#             DataFromOneFile['Metadata_Barcode'].replace('exp180', str(inpF['plate-id']), inplace=True)
#             print(DataFromOneFile.shape)   
#             DataFromFolder = pd.concat([DataFromFolder, DataFromOneFile.iloc[:,:]], ignore_index=True, sort=False, axis=0, join = 'outer')
#             print('{}/ImageMeansFrame{}.csv'.format(OutputDir, inpF['plate-id']))
            
            


       featrue_names
0     featICF_nuclei
1      featICF_cells
2  featICF_cytoplasm


In [4]:
now = datetime.datetime.now()
print ('Current date and time : ')
print (now.strftime('%Y-%m-%d %H:%M:%S'))

Current date and time : 
2023-03-29 15:29:58


In [38]:
# medianImageAllPlates = pd.DataFrame()
# meanImageAllPlates = pd.DataFrame()
# wellmismatch = 0

# DataFrameDictionary = {}s
ObjectList = ['featICF_nuclei', 'featICF_cells', 'featICF_cytoplasm']
nuclei = pd.read_csv(BaseDir+f'/{ObjectList[0]}.csv', sep=',').add_prefix('Nuclei_').reset_index()
cytoplasm = pd.read_csv(BaseDir+f'/{ObjectList[1]}.csv', sep=',').add_prefix('Cytoplasm_').reset_index()
cells = pd.read_csv(BaseDir+f'/{ObjectList[2]}.csv', sep=',').add_prefix('Cells_').reset_index()
# step 1: Take the mean values of multiple nuclei belonging to one cell
nuclei = nuclei.groupby(["Nuclei_Metadata_Barcode","Nuclei_Metadata_Well","Nuclei_Parent_cells", 'Nuclei_Metadata_Plane'], as_index=False).mean(numeric_only=True)
# step 2: merge nuclei and cytoplasm objects
df = pd.merge(nuclei, cytoplasm, how='left', left_on=['Nuclei_Metadata_Well', 'Nuclei_Metadata_Plane',"Nuclei_Parent_cells", 'Nuclei_Metadata_Barcode'],
                right_on = ['Cytoplasm_Metadata_Well','Cytoplasm_Metadata_Plane', "Cytoplasm_ObjectNumber", 'Cytoplasm_Metadata_Barcode'])
# step 3: join cells objects
df = pd.merge(df, cells, how='left', left_on=['Cytoplasm_Metadata_Well','Cytoplasm_Metadata_Plane',"Cytoplasm_ObjectNumber", 'Cytoplasm_Metadata_Barcode'],
                right_on = ['Cells_Metadata_Well','Cells_Metadata_Plane',"Cells_ObjectNumber", 'Cells_Metadata_Barcode'])

#------------------- format feature names to BROAD names --------------------------------#
first_column = df.pop('Cells_Metadata_Barcode')
second_column = df.pop('Cells_Metadata_Well')
third_column = df.pop('Cells_Metadata_Plane')

df.insert(0, 'Metadata_Barcode', first_column)
df.insert(1, 'Metadata_Well', second_column)
df.insert(2, 'Metadata_Plane', third_column)
nameargs = ['Nuclei', 'Cytoplasm', 'Cells']
for name in nameargs:
    df = df.loc[:,~df.columns.str.contains(name+'_Metadata_', case=True)]



# ------------------ add image indentifyer for metdata and QC -----------------------------#
df = df.loc[:,~df.columns.str.contains('Unnamed|index|Cytoplasm_Meta|Cells_Meta|Nuclei_Meta|FileName|PathName|_ImageNumber')]

NrOfObjects = df.shape[0]
Wells = sorted(list(set(df['Metadata_Well'])))
NrOfWells = len(Wells)
Rows = sorted(list(set([w[0] for w in Wells])))
print(*Rows)
NrOfRows = len(Rows)
Columns = sorted(list(set([w[1:] for w in Wells])))
NrOfColumns = len(Columns)
print(*Columns)
print('Plate complete: {}'.format(NrOfRows*NrOfColumns==NrOfWells))

AllWells =[]
for R in Rows:
    for C in Columns:
        RC = R + C
        # print(RC)
        AllWells += [RC]
# print(len(AllWells))
print('Number of wells: {} (full plate is {})'.format(NrOfWells, len(AllWells)))
del nuclei
del cytoplasm
del cells
gc.collect()
df.to_parquet('{}/spheroidSingleCell.parquet'.format(OutputDir))


A
01 02 03 04 05
Plate complete: True
Number of wells: 5 (full plate is 5)


In [ ]:
now = datetime.datetime.now()
print ('Current date and time : ')
print (now.strftime('%Y-%m-%d %H:%M:%S'))